In [ ]:
import csv
from datetime import datetime, timedelta, timezone
import pytz
import sys
from io import StringIO

# Helper function to parse datetime strings with timezone
def parse_datetime(dt_str):
    try:
        return datetime.fromisoformat(dt_str.replace("Z", "+00:00"))
    except Exception as e:
        raise ValueError(f"Failed to parse datetime string '{dt_str}': {str(e)}")

# Function to calculate max achievable SoC (charging)
def calculate_max_soc(start_time, end_time, soc_start, consumption_capacity, resolution, efficiency):
    soc = soc_start
    current_time = start_time
    step_duration = resolution.total_seconds() / 3600

    while current_time < end_time:
        power = 0
        for cc in consumption_capacity:
            if cc["start"] <= current_time < cc["end"]:
                power = cc["value"]
                break
        energy = (power / 1000) * step_duration * efficiency
        soc += energy
        current_time += resolution
    return soc

# Function to calculate min achievable SoC (discharging)
def calculate_min_soc(start_time, end_time, soc_start, production_capacity, resolution, efficiency):
    soc = soc_start
    current_time = start_time
    step_duration = resolution.total_seconds() / 3600
    while current_time < end_time:
        power = 0
        for pc in production_capacity:
            if pc["start"] <= current_time < pc["end"]:
                power = pc["value"]
                break
        energy = (power / 1000) * step_duration * efficiency
        soc -= energy
        current_time += resolution
    return soc

# Function to analyze and categorize infeasibility
def analyze_and_categorize_infeasibility(job_id, kwargs_str):
    try:
        eval_globals = {"datetime": __import__("datetime"), "timezone": timezone}
        kwargs = eval(kwargs_str, eval_globals)

        start = kwargs["start"]
        end = kwargs["end"]
        resolution = kwargs["resolution"]
        flex_model = kwargs.get("flex_model", {})
        flex_context = kwargs.get("flex_context", {})

        # Extract flex_model fields
        soc_at_start = flex_model.get("soc-at-start", 0)
        soc_unit = flex_model.get("soc-unit", "kWh")
        soc_min = flex_model.get("soc-min", 0)
        soc_max = flex_model.get("soc-max", float("inf"))
        soc_minima = flex_model.get("soc-minima", [])
        soc_maxima = flex_model.get("soc-maxima", [])
        consumption_capacity = flex_model.get("consumption-capacity", [])
        production_capacity = flex_model.get("production-capacity", [])
        roundtrip_efficiency = flex_model.get("roundtrip-efficiency", 1.0)
        efficiency = roundtrip_efficiency if roundtrip_efficiency <= 1 else roundtrip_efficiency / 100
        charging_efficiency = efficiency ** 0.5
        discharging_efficiency = efficiency ** 0.5

        # Convert nested datetime strings to datetime objects
        for minima in soc_minima:
            minima["start"] = parse_datetime(minima["start"])
            minima["end"] = parse_datetime(minima["end"])
        for maxima in soc_maxima:
            maxima["start"] = parse_datetime(maxima["start"])
            maxima["end"] = parse_datetime(maxima["end"])
        for cc in consumption_capacity:
            cc["start"] = parse_datetime(cc["start"])
            cc["end"] = parse_datetime(cc["end"])
            cc["value"] = float(cc["value"].split()[0])
        for pc in production_capacity:
            pc["start"] = parse_datetime(pc["start"])
            pc["end"] = parse_datetime(pc["end"])
            pc["value"] = float(pc["value"].split()[0])

        categories = {
            "Initial SoC Violations": [],
            "SoC Minima/Maxima Conflicts": [],
            "Power Capacity Constraints": [],
            "Insufficient Charging Capacity": [],
            "Insufficient Discharging Capacity": [],
            "Unspecified Issues": []
        }

        # Check 1: Initial SoC vs global soc-min/soc-max
        if soc_at_start < soc_min:
            categories["Initial SoC Violations"].append(
                f"Initial SoC ({soc_at_start} {soc_unit}) is below global soc-min ({soc_min} {soc_unit})."
            )
        if soc_at_start > soc_max:
            categories["Initial SoC Violations"].append(
                f"Initial SoC ({soc_at_start} {soc_unit}) exceeds global soc-max ({soc_max} {soc_unit})."
            )

        # Check 2: Initial SoC vs soc-minima at start
        for minima in soc_minima:
            if minima["start"] <= start < minima["end"] and soc_at_start < minima["value"]:
                categories["SoC Minima/Maxima Conflicts"].append(
                    f"Initial SoC ({soc_at_start} {soc_unit}) at {start} is below soc-minima "
                    f"({minima['value']} {soc_unit}) from {minima['start']} to {minima['end']}."
                )

        # Check 3: Initial SoC vs soc-maxima at start
        for maxima in soc_maxima:
            if maxima["start"] <= start < maxima["end"] and soc_at_start > maxima["value"]:
                categories["SoC Minima/Maxima Conflicts"].append(
                    f"Initial SoC ({soc_at_start} {soc_unit}) at {start} exceeds soc-maxima "
                    f"({maxima['value']} {soc_unit}) from {maxima['start']} to {maxima['end']}."
                )

        # Check 4: Ability to meet soc-minima with zero capacity at start
        for minima in soc_minima:
            if minima["start"] <= start < minima["end"] and soc_at_start < minima["value"]:
                can_charge = False
                for cc in consumption_capacity:
                    if cc["start"] <= start < cc["end"] and cc["value"] > 0:
                        can_charge = True
                        break
                if not can_charge:
                    categories["Power Capacity Constraints"].append(
                        f"Cannot charge to meet soc-minima ({minima['value']} {soc_unit}) from "
                        f"{minima['start']} to {minima['end']} because consumption-capacity is 0 W "
                        f"at start ({start})."
                    )

        # Check 5: Ability to discharge if above soc-maxima with zero capacity at start
        for maxima in soc_maxima:
            if maxima["start"] <= start < maxima["end"] and soc_at_start > maxima["value"]:
                can_discharge = False
                for pc in production_capacity:
                    if pc["start"] <= start < pc["end"] and pc["value"] > 0:
                        can_discharge = True
                        break
                if not can_discharge:
                    categories["Power Capacity Constraints"].append(
                        f"Cannot discharge to meet soc-maxima ({maxima['value']} {soc_unit}) from "
                        f"{maxima['start']} to {maxima['end']} because production-capacity is 0 W "
                        f"at start ({start})."
                    )

        # Check 6: Insufficient charging capacity to meet future soc-minima
        for minima in soc_minima:
            if minima["start"] > start and minima["end"] <= end:
                max_soc = calculate_max_soc(start, minima["start"], soc_at_start, consumption_capacity, resolution, charging_efficiency)
                if max_soc < minima["value"]:
                    categories["Insufficient Charging Capacity"].append(
                        f"Cannot reach soc-minima ({minima['value']} {soc_unit}) by {minima['start']} "
                        f"from initial SoC ({soc_at_start} {soc_unit}) at {start}. "
                        f"Max achievable SoC is {max_soc:.2f} {soc_unit} due to insufficient charging capacity."
                    )

        # Check 7: Insufficient discharging capacity to meet soc-maxima transitions
        sorted_maxima = sorted([m for m in soc_maxima if m["start"] <= end and m["end"] > start], key=lambda x: x["start"])
        for i in range(1, len(sorted_maxima)):
            prev_max = sorted_maxima[i - 1]
            curr_max = sorted_maxima[i]
            if prev_max["end"] == curr_max["start"] and prev_max["value"] > curr_max["value"]:
                min_soc = calculate_min_soc(start, curr_max["start"], soc_at_start, production_capacity, resolution, discharging_efficiency)
                if min_soc > curr_max["value"]:
                    categories["Insufficient Discharging Capacity"].append(
                        f"Cannot reduce SoC from {prev_max['value']} {soc_unit} to meet soc-maxima "
                        f"({curr_max['value']} {soc_unit}) by {curr_max['start']} from initial SoC "
                        f"({soc_at_start} {soc_unit}) at {start}. Min achievable SoC is {min_soc:.2f} {soc_unit} "
                        f"due to insufficient discharging capacity."
                    )

        # If no specific reason found, categorize as unspecified
        if not any(categories.values()):
            categories["Unspecified Issues"].append(
                "Infeasibility detected, but no specific constraint violation identified. "
                "Possible issues: insufficient power capacity, conflicting soc targets, or solver limitations."
            )

        result = {
            "categories": {k: v for k, v in categories.items() if v},
            "flex_model": flex_model,
            "flex_context": flex_context,
            "start": start,
            "end": end
        }
        return result

    except Exception as e:
        import traceback
        tb = traceback.format_exc()
        return {"categories": {"Error": [f"Error analyzing job {job_id}: {str(e)}\nTraceback: {tb}"]}, "flex_model": {}, "flex_context": {}, "start": None, "end": None}

def process_csv_sensor_x(file_path, sensor_id_filter=None, output_file="Ard_sensor_analysis.txt"):
    category_counts = {
        "Initial SoC Violations": 0,
        "SoC Minima/Maxima Conflicts": 0,
        "Power Capacity Constraints": 0,
        "Insufficient Charging Capacity": 0,
        "Insufficient Discharging Capacity": 0,
        "Unspecified Issues": 0,
        "Error": 0
    }
    jobs_analyzed = 0

    with open(output_file, "w", encoding="utf-8") as output_f:
        def dual_print(*args, **kwargs):
            old_stdout = sys.stdout
            sys.stdout = string_buffer = StringIO()
            print(*args, **kwargs)
            sys.stdout = old_stdout
          
            output = string_buffer.getvalue()
            print(output, end="")
            output_f.write(output)
            string_buffer.close()

        with open(file_path, "r") as f:
            reader = csv.DictReader(f)
            for row in reader:
                job_id = row["Job ID"]
                sensor_id = int(row["ID"])
                error = row["Error"]
                kwargs_str = row["All kwargs"]

                if "InfeasibleProblemException" in error and (sensor_id_filter is None or sensor_id == sensor_id_filter):
                    dual_print(f"\nAnalyzing Job ID: {job_id} (Sensor ID: {sensor_id})")
                    result = analyze_and_categorize_infeasibility(job_id, kwargs_str)
                    categories = result["categories"]
                    flex_model = result["flex_model"]
                    flex_context = result["flex_context"]
                    start = result["start"]
                    end = result["end"]

                    dual_print(f"Start: {start}")
                    dual_print(f"End: {end}")

                    dual_print("Categorized Reasons for Infeasibility:")
                    for category, reasons in categories.items():
                        dual_print(f"{category}:")
                        for reason in reasons:
                            dual_print(f"  - {reason}")
                        category_counts[category] += 1

                    def dual_print_dict(label, d, indent="  "):
                        dual_print(f"{label}:")
                        for key, value in d.items():
                            if isinstance(value, list):
                                dual_print(f"{indent}- {key}:")
                                for item in value:
                                    dual_print(f"{indent}  - {item}")
                            else:
                                dual_print(f"{indent}- {key}: {value}")

                    dual_print_dict("Flex Model", flex_model)
                    dual_print_dict("Flex Context", flex_context)

                    jobs_analyzed += 1

            dual_print(f"\nTotal jobs analyzed{' for Sensor ID ' + str(sensor_id_filter) if sensor_id_filter is not None else ''}: {jobs_analyzed}")
            dual_print(f"Summary of Infeasibility Categories{' for Sensor ID ' + str(sensor_id_filter) if sensor_id_filter is not None else ''}:")
            for category, count in category_counts.items():
                dual_print(f"{category}: {count} occurrences")

    print(f"Analysis has been written to {output_file}")

# Example usage
if __name__ == "__main__":
    csv_file_path = "Ard-failed-jobs.csv"
    # For a specific sensor (e.g., 99):
    process_csv_sensor_x(csv_file_path, sensor_id_filter=99)
    # For all sensors:
    # process_csv_sensor_x(csv_file_path)